In [34]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

import time
from time import sleep
import re

import numpy as np
import pandas as pd

In [2]:
keywords = ["邊坡", "崩塌", "復建"]
orgkws = ["水土保持局臺北分局", "羅東林區管理處 ", "第四區養護工程處", "宜蘭縣"]

In [3]:
driver = webdriver.PhantomJS(executable_path=r"D:\PyProjects\projcrawler\phantomjs-2.1.1-windows\bin\phantomjs.exe")
bidSystemUrl = "https://web.pcc.gov.tw/tps/pss/tender.do?method=goSearch&searchMode=common&searchType=basic"
browse = driver.get(bidSystemUrl)
homepage = driver.page_source
query = driver.find_element_by_name("tenderName")
query.clear()
query.send_keys("復建")
queryCate = driver.find_element_by_css_selector("#radProctrgCate1")
queryCate.click()

C:\Users\User\anaconda3\envs\py38\lib\site-packages\selenium\webdriver\phantomjs\webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


In [8]:
tender_table

[<tr bgcolor="#FFFFFF" onmouseout="outcss(this,'#FFFFFF');" onmouseover="overcss(this);">
 <!-- 項次 -->
 <td align="center">1 </td>
 <!-- 機關名稱 -->
 <td align="left">新北市坪林區公所 </td>
 <!-- 標案案號&標案名稱 -->
 <td align="left">
 												111011201
 												
 												<br/>
 <a href="../tpam/main/tps/tpam/tpam_tender_detail.do?searchMode=common&amp;scope=F&amp;primaryKey=53720251" title="111年度坪林區天然災害搶災搶險及復建工程開口契約">
 <u>111年度坪林區天然災害搶災搶險及復建工程開口契約</u>
 </a>
 													
 													
 													
 													
 												 
 												</td>
 <!-- 傳輸次數 -->
 <td align="center">
 <a href="../tpam/main/tps/tpam/tpam_tender_detail.do?searchMode=common&amp;scope=F&amp;primaryKey=53720251" title="03">
 <u>03</u>
 </a>
 														
 														
 													 
 												</td>
 <!-- 招標方式 -->
 <td align="left">公開招標</td>
 <!-- 標的分類 -->
 <td align="left">工程類</td>
 <!-- 公告日期 -->
 <td align="left">111/02/10
 </td>
 <!-- 截止投標 -->
 <td align="left">111/02/16
 </td>
 <!-- 預算金額 -->
 <td al

In [74]:
def get_tender_by_kw(kws, loc):
    driver = webdriver.PhantomJS(executable_path=r"D:\PyProjects\projcrawler\phantomjs-2.1.1-windows\bin\phantomjs.exe")
    tender_list = []
    for k in kws:
        bidSystemUrl = "https://web.pcc.gov.tw/tps/pss/tender.do?method=goSearch&searchMode=common&searchType=basic"
        browse = driver.get(bidSystemUrl)
        homepage = driver.page_source
        query = driver.find_element_by_name("tenderName")
        query.clear()
        query.send_keys(k)
        queryCate = driver.find_element_by_css_selector("#radProctrgCate1")
        queryCate.click()
        query_loc = driver.find_element_by_name("orgName")
        query_loc.send_keys(loc)
        query.send_keys(Keys.RETURN)
        sleep(1) #等頁面回傳
        bidpage = driver.page_source
        soup = BeautifulSoup(bidpage, 'html.parser')
        tender_table = soup.find_all("tr", onmouseover="overcss(this);")
        tender_list += tender_table
    return list(set(tender_list))

def get_tender_by_org(orgs):
    driver = webdriver.PhantomJS(executable_path=r"D:\PyProjects\projcrawler\phantomjs-2.1.1-windows\bin\phantomjs.exe")
    tender_list = []
    for o in orgs:
        bidSystemUrl = "https://web.pcc.gov.tw/tps/pss/tender.do?method=goSearch&searchMode=common&searchType=basic"
        browse = driver.get(bidSystemUrl)
        homepage = driver.page_source
        query = driver.find_element_by_name("orgName")
        query.clear()
        query.send_keys(o)
        queryCate = driver.find_element_by_css_selector("#radProctrgCate1")
        queryCate.click()
        query.send_keys(Keys.RETURN)
        sleep(1) #等頁面回傳
        bidpage = driver.page_source
        soup = BeautifulSoup(bidpage, 'html.parser')
        tender_table = soup.find_all("tr", onmouseover="overcss(this);")
        tender_list += tender_table
    return list(set(tender_list))

def get_tender_info(tender):
    return {
        "機關名稱": "".join(tender.find_all("td")[1].text.split()), 
        "標案案號&標案名稱": "".join(tender.find_all("td")[2].text.split()), 
        "傳輸次數": "".join(tender.find_all("td")[3].text.split()), 
        "招標方式": "".join(tender.find_all("td")[4].text.split()), 
        "採購性質": "".join(tender.find_all("td")[5].text.split()), 
        "公告日期": "".join(tender.find_all("td")[6].text.split()), 
        "截止投標": "".join(tender.find_all("td")[7].text.split()), 
        "預算金額": "".join(tender.find_all("td")[8].text.split()), 
        "url": tender.find_all("td")[2].find('a').get('href').replace("..", "https://web.pcc.gov.tw/tps")
           }

In [10]:
t_ks = []
for loc in ["臺北", "新北", "桃園", "新竹", "宜蘭"]:
    t = get_tender_by_kw(keywords, loc)
    t_ks += t
t_orgs = get_tender_by_org(orgkws)
t_list = list(set(t_ks+t_orgs))

In [11]:
len(t_ks), len(t_orgs), len(t_list)

(5, 6, 11)

In [76]:
get_tender_info(t_list[1])

{'機關名稱': '交通部公路總局第四區養護工程處',
 '標案案號&標案名稱': '1104B133台7線85k+400~87k+800、台7甲線0k+000~1k+500、7k+400~9k+400路面改善工程',
 '傳輸次數': '04',
 '招標方式': '公開招標',
 '採購性質': '工程類',
 '公告日期': '111/02/10',
 '截止投標': '111/02/21',
 '預算金額': '15,978,909',
 'url': 'https://web.pcc.gov.tw/tps/tpam/main/tps/tpam/tpam_tender_detail.do?searchMode=common&scope=F&primaryKey=53719607'}

In [1]:
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup

import json

from datetime import datetime, timedelta
import os

In [2]:
def get_proj_list_by_date(date):
    unit_keywords = ['宜蘭', '羅東', '第四區養護工程處']
    date_url = f"https://pcc.g0v.ronny.tw/api/listbydate?date={date}"
    
    r = requests.get(date_url)
    d = json.loads(r.text)
    p = [i for i in d['records'] if (i['brief']['type'] == '公開招標公告') and ('工程類' in i['brief']['category'])]
    
    projs = []
    for k in unit_keywords:
        for i in p:
            try:
                if k in i['unit_name']:
                    projs.append(i)
            except:
                continue
    return projs

def get_proj_info(unit_id, job_number):
    proj_url = f"https://pcc.g0v.ronny.tw/api/tender?unit_id={unit_id}&job_number={job_number}"
    r = requests.get(proj_url)
    d = json.loads(r.text)
    return {
        "unit_name": d['records'][0]['unit_name'], 
        "proj_name": d['records'][0]['brief']['title'], 
        "proj_type": d['records'][0]['brief']['type'], 
        "proj_category": d['records'][0]['brief']['category'], 
        "proj_budget": d['records'][0]['detail']['採購資料:預算金額'], 
        "proj_loc": d['records'][0]['detail']['其他:履約地點'], 
        "proj_pub_time":d['records'][0]['detail']['招標資料:公告日'], 
        "proj_end_time":d['records'][0]['detail']['領投開標:截止投標'], 
        "proj_open_time":d['records'][0]['detail']['領投開標:開標時間'], 
        "url": d['records'][0]['detail']['url']
           }

In [3]:
# Monday is 0, Sunday is 6
weekday = datetime.today().weekday()
day_list = [i.strftime('%Y%m%d') for i in pd.date_range(datetime.today()-timedelta(days=7+weekday), datetime.today()-timedelta(days=2+weekday))][:-1]
day_list

['20210726', '20210727', '20210728', '20210729', '20210730']

In [4]:
proj_df = pd.DataFrame()
for date in day_list:
    projs = get_proj_list_by_date(date)
    for proj in projs:
        temp_df = pd.DataFrame()
        proj_info = get_proj_info(proj['unit_id'], proj['job_number'])
        for k, v in proj_info.items():
            temp_df[k] = [v]
        proj_df = pd.concat([proj_df, temp_df], ignore_index=True)
proj_df.drop_duplicates(keep='last', ignore_index=True, inplace=True)
proj_df.rename(columns={"unit_name":"機關名稱", "proj_name":"標案名稱", "proj_type":"招標方式", "proj_category":"標的分類", 
                        "proj_budget":"預算金額", "proj_loc":"履約地點", "proj_pub_time":"公告日", "proj_end_time":"截止投標", 
                        "proj_open_time":"開標時間"}, inplace=True)

In [5]:
saved_name = os.path.join("c:", "Users", "User", "Desktop", "標案資料", f"{datetime.today().strftime('%Y%m%d')}.xlsx")
proj_df.to_excel(saved_name, index=None)